In [7]:
import cmath
import numpy as np
import math
pi = math.pi

import itertools

In [2]:
# PySCF
import pyscf
from pyscf import gto, scf, mcscf, cc

The ```PySCF``` package is already included in ```Qiskit``` and can be used as a part of it. But I don't like this inclusion and prefer to use pyscf directly. The drawback of such approach is that one needs to extract one- and two-body integrals from pyscf and write them in a form which can be understood by ```Qiskit```. This should be done since it is convenient to use ```Qiskit``` for Hamiltonian construction as in the second quantization form as well as in the form appropriate for the quantum computer (sum of the Pauli strings). The extraction of the 1- and 2-body integrals is done by the procedure, which was cutted from ```Qiskit``` and placed into ```integrals_mod.py``` file.

In [3]:
from integrals_mod import *

In many cases it will be convenient to write the state of the quantum computer as a vector and to transform operations on qubits into matrices-vector multiplications. This will allow to check hether everything works correctly. The related functions are planned to place in ```classical_mod.py```.

In [4]:
import classical_mod

The file ```quantum_mod.py``` contains functions which are used for simulation of the quantum computer or even launch the code on the real quantum computer.

In [5]:
from quantum_mod import *

In [6]:
# Qiskit
from qiskit import *
from qiskit_nature.problems.second_quantization.electronic.builders.fermionic_op_builder import build_ferm_op_from_ints

from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper

from qiskit.quantum_info import Statevector

We start from specifying the system of interest, lets start from H4 molecule.

In [9]:
sz_tot = 0 # even number of electrons
#sz_tot = 1 # odd number of electrons

mol = gto.M(
        #atom = 'H 0 0 0; H 0 0 0.735',
        #atom = 'Li 0 0 0; H 0 0 1.5',
        atom = 'H 0 0 0; H 0 0 1.24; H 0 1.24 1.24; H 0 1.24 0',
        basis = 'sto-3g',
        #basis = 'ccpvdz',
        #basis = 'ccpvtz',
        #basis = 'cc-pVQZ',
        #basis = 'cc-pV5Z',
        spin = sz_tot, verbose=0)

Let us see some info about the system

In [11]:
nao = mol.nao
nso = 2*nao
print("Number of atomic orbitals = ", nao, flush=True)
print("Number of spin orbitals = ", nso, flush=True)

nelectron = mol.nelectron
na = mol.nelec[0]
nb = mol.nelec[1]
print("Number of electrons = ", nelectron, "alpha = ", na,"beta = ", nb)

Number of atomic orbitals =  4
Number of spin orbitals =  8
Number of electrons =  4 alpha =  2 beta =  2


We work in the nonrelativistic formalism but with spin. The presence of spin is accounted for only via the Pauli principle, e.g., two particles with exactly the same quantum numbers can't occupy the same state.

The spin projection can be either +1/2 or -1/2. Let us assign the electrons with the spin +1/2 to the alpha class and electrons with -1/2 to the beta class. The number of electrons in these classes are equal na and nb, respectively. Note that since ```sz_tot = 0.5*(na - nb)``` is a good quantum number, the number of particles in each class can't be changed.

In [13]:
# Hartree-Fock
mf = scf.RHF(mol).run()
E_HF = mf.e_tot
print("\nHartree-Fock energy = ", E_HF, "[Ha]", flush=True)


# Nucleus interaction energy
E_Nuc = mf.energy_nuc()
print("Nucleus interaction energy = ",E_Nuc, "[Ha]")


# Complete CI in a given basis
mc_fci = mcscf.CASCI(mf, nao, nelectron).run(verbose=0)
E_FCI = mc_fci.e_tot
print("FCI electronic energy = ", E_FCI, "[Ha]")
print("Correlations = ", f'{abs(E_FCI - E_HF)*1.e3: <.3f}', "[mHa]",
      flush=True)


Hartree-Fock energy =  -1.699765101353842 [Ha]
Nucleus interaction energy =  2.3105471227918004 [Ha]
FCI electronic energy =  -1.9699338641352195 [Ha]
Correlations =  270.169 [mHa]


We are interested in the difference between the Hartree-Fock energy and the FCI (Full Configuration Interaction) energy. FCI energy is the most accurate value which can be obtained in the given basis. If the difference between HF and FCI energies is smaller than the required accuracy, than there is no need to go beyond HF and use quantum computer and etc. Here and throught we assume that the accuracy on the level of 1 kcal/mol (1.594 mHa) is required.

In [14]:
# CCSD
mc_ccsd = cc.CCSD(mf).run(verbose=0)
E_CCSD = mc_ccsd.e_tot
print("\nCCSD electronic energy = ", E_CCSD, "[Ha]")
print("diff with FCI = ", f'{abs(E_FCI - E_CCSD)*1.e3: <.3f}', "[mHa]",
      flush=True)


CCSD electronic energy =  -1.7876561755669462 [Ha]
diff with FCI =  182.278 [mHa]


While FCI calculation is extremely expensive, CCSD (Coupled-Cluster Single Double) can be performed even for systems with relatively large number of electrons. Tge growth of it's complexety is about N^7, where N is the number of the basis functions. The systems, where CCSD approach provides the energy differing from the FCI one by more than 1 1.594 mHa are in the scope of our interest. The H4 molecule is exactly the case.

In [16]:
# The one- and two-body integrals are calculated and 
# transformed to the form required for the Qiskit package
one_b_int, two_b_int = integrals(mol, mc_fci)

The 1- and 2-body integrals are used for the construction of the Hamiltonian in the second quantization picture

In [17]:
H_op = build_ferm_op_from_ints(one_body_integrals=one_b_int,
                               two_body_integrals=two_b_int)

To proceed further, we have to chose the way of mapping the second quantization operators to the operations on qubits. Parity and Bravyi-Kitaev mappings allow to drop two qubits from the Hamiltonian and consider the problem of the smaller size without loosing any information. For transparancy, however, it is more convenient to use Jordan-Wigner mapping in which the occupied spin-orbital is related to the |1> state of the qubit and vacant to the |0> one. In this encoding the number of qubits equals the number of spin-orbitals

In [19]:
mapper = JordanWignerMapper()
reduction = False

qubit_converter = QubitConverter(mapper=mapper,
                                 two_qubit_reduction=reduction)

H_q = qubit_converter.convert(H_op, num_particles=nelectron)

Nq = H_q.num_qubits
print( "\n Nq = ", Nq, flush=True )


 Nq =  8


For the moment, the most accurate approximation of the ground-state wave function can be taken from HF calculation. Note that though CCSD approach provides the energy which is closer to the FCI one, it do not provide the wave function. Therefore, we will use the HF wave function as an input to the quantum computer. This is equivalent to filling ```na``` orbitals in the alpha class and ```nb``` orbitals in the beta class. For the H4 molecule in the STO-3G basis there are 8 spin-orbitals with sets of numbers 0..3 and 4..7 numerating the alpha and beta classes, respectively.

In [21]:
# Initial guess of the ground state
occ_hf_a = [i for i in range(na)]
occ_hf_b = [nao + i for i in range(nb)] 
occ_hf = occ_hf_a + occ_hf_b

print("occupied orbitals = ", occ_hf)

occupied orbitals =  [0, 1, 4, 5]


Let us also create the circuit for preparing the HF wave function and translate it to the vector. Note that in ```Qiskit``` the qubits are numerated from the right, e.g., q7 q6 ... q0. So, the rightmost qubit is related to the 0 orbital.

In [23]:
psi_HF_circ = QuantumCircuit(Nq, Nq)
lbl = "0"*Nq
for o in occ_hf:
  psi_HF_circ.x(o)
  lbl = lbl[:Nq-1-o] + "1" + lbl[Nq-o:]
  
print(lbl)
psi_HF_vec = Statevector.from_label(lbl).data

00110011


Let us perform a simple check that our 1- and 2-body integrals were trasferred to the Qiskit correctly and that the HF wave function is also correct

In [24]:
H_mtrx = H_q.to_matrix().real # for classical tests

print("< hf | H | hf > = ", 
      psi_HF_vec.conj().T.dot( H_mtrx.dot(psi_HF_vec) ).item(0).real + E_Nuc,
      "should be", E_HF)

< hf | H | hf > =  -1.6997651013538384 should be -1.699765101353842


Now we will find all possible variants of populating the orbitals from the alpha class by ```na``` electrons

In [25]:
# Numbers of alpha orbitals
orbs_a = [i for i in range(nao)]

# list of all possible occupations of the alpha class
list_a = list(itertools.combinations(orbs_a,na))

print(list_a)

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


The same for the beta set

In [27]:
orbs_b = [nao+i for i in range(nao)]
list_b = list(itertools.combinations(orbs_b,nb))
print(list_b)

[(4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)]


Currently the alpha and beta sets are occupied as follows

In [28]:
print(occ_hf_a)
print(occ_hf_b)

[0, 1]
[4, 5]


Here we empose the restriction that the state, which is described by the occupation numbers from ```list_a``` can be obtained only by annihilating the electrons in ```occ_hf_a```. The same for the beta set. 

For each element in ```list_a``` let us find which electron from ```occ_hf_a``` should be annihilated and where it should be created. 

In [39]:
for occ_a in list_a:
    ann_a = [a for a in occ_hf_a if not a in occ_a]
    crt_a = [a for a in occ_a if not a in occ_hf_a]
    
    print("To create", occ_a, "from", occ_hf_a ,"ann", ann_a, "crt",crt_a)

To create (0, 1) from [0, 1] ann [] crt []
To create (0, 2) from [0, 1] ann [1] crt [2]
To create (0, 3) from [0, 1] ann [1] crt [3]
To create (1, 2) from [0, 1] ann [0] crt [2]
To create (1, 3) from [0, 1] ann [0] crt [3]
To create (2, 3) from [0, 1] ann [0, 1] crt [2, 3]


The number of creation (or annihilation) operators required equals to the number of excitations. The Doubles are related to exactly 2 excitations in both alpha and beta sets.

In [43]:
ann = []
crt = []
for occ_a in list_a:
    ann_a = [a for a in occ_hf_a if not a in occ_a]
    crt_a = [a for a in occ_a if not a in occ_hf_a]
    for occ_b in list_b:
        ann_b = [b for b in occ_hf_b if not b in occ_b]
        crt_b = [b for b in occ_b if not b in occ_hf_b]

        # if the number of excitation != 2 we proceed further
        if len(crt_a + crt_b) != 2:
            continue

        ann.append( ann_a + ann_b )
        crt.append( crt_a + crt_b )            

        print("ann:",ann[-1],"crt:",crt[-1])

ann: [4, 5] crt: [6, 7]
ann: [1, 5] crt: [2, 6]
ann: [1, 5] crt: [2, 7]
ann: [1, 4] crt: [2, 6]
ann: [1, 4] crt: [2, 7]
ann: [1, 5] crt: [3, 6]
ann: [1, 5] crt: [3, 7]
ann: [1, 4] crt: [3, 6]
ann: [1, 4] crt: [3, 7]
ann: [0, 5] crt: [2, 6]
ann: [0, 5] crt: [2, 7]
ann: [0, 4] crt: [2, 6]
ann: [0, 4] crt: [2, 7]
ann: [0, 5] crt: [3, 6]
ann: [0, 5] crt: [3, 7]
ann: [0, 4] crt: [3, 6]
ann: [0, 4] crt: [3, 7]
ann: [0, 1] crt: [2, 3]


Now we write the double excitation as the following operator

$$e^{t(T_2 - T_2^\dagger)}$$

with 

$$T_2 = a^\dagger_p a^\dagger_q a_r a_s$$

[p,q] and [r,s] stand for the elements of ```ann``` and ```crt```, respectively, and *t* is the cluster amplitude which is to be find via variational approach. 

To transform this operator to the gates, one needs to express it as a sum of PS (Pauli Strings). For this purpose the ```Qiskit``` package can be used.

In [42]:
D_q = []
for i in range(len(ann)):
    if len(crt[i]) != 2:
        continue

    T = FermionicOp("", register_length=Nq)
    for o in crt[i]:
        T @= FermionicOp("+_"+str(o), register_length=Nq)
    for o in ann[i]:
        T @= FermionicOp("-_"+str(o), register_length=Nq)
    T -= ~T

    T_q = qubit_converter.convert(T.reduce(), num_particles=nelectron)

    D_q.append(T_q)

As an example, for the last double excitation one obtains 

In [60]:
indx = np.random.randint(0, high=len(ann), dtype=int)

print("ann",ann[indx],"crt",crt[indx])
print("\nT - T^+ = ", D_q[indx])

ann [0, 1] crt [2, 3]

T - T^+ =  -0.125j * IIIIYXXX
+ -0.125j * IIIIXYXX
+ 0.125j * IIIIXXYX
+ -0.125j * IIIIYYYX
+ 0.125j * IIIIXXXY
+ -0.125j * IIIIYYXY
+ 0.125j * IIIIYXYY
+ 0.125j * IIIIXYYY


Now we transform

$$e^{t(T_2 - T_2^\dagger)} = e^{t\sum_i c_i PS_i} \approx \Pi_i e^{t c_i PS_i}$$

For each PS in the product one needs to construct the circuit, which for the arbitrary choice of the PS is given by

In [72]:
indx_ps = np.random.randint(0, high=len(D_q[indx]), dtype=int)
#t_ang = 

PS, coef = D_q[indx][indx_ps].primitive.to_list()[0]

print("PS = ", PS, coef)

#ang = -1j * coef * t_ang

#print( exp_alpha_PS_circ(ang.real, PS) ) 

PS =  IIIIYXYY
